In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from Load import json_to_df
import tensorflow_hub as hub
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer

Using TensorFlow backend.


In [2]:
df_music = pd.read_csv("data\\music_reviews(subset).csv")

In [3]:
df_music_dev = pd.read_csv("data\\music_reviews_dev(subset).csv")

In [4]:
df_music=df_music.replace({"sentiment":{"positive":1,"negative":0}})
df_music_dev=df_music_dev.replace({"sentiment":{"positive":1,"negative":0}})

In [5]:
df_music = df_music.replace(np.nan, '', regex=True)
df_music['review_headline'] = df_music.review_headline + ' ' + df_music.review_body

In [6]:
df_music_dev = df_music_dev.replace(np.nan, '', regex=True)
df_music_dev['review_headline'] = df_music_dev.review_headline + ' ' + df_music_dev.review_body

In [7]:
train_text = df_music["review_headline"].tolist()
train_text = [' '.join(t.split()[0:150]) for t in train_text]
train_text = np.array(train_text, dtype=object)[:, np.newaxis]
train_label = df_music["sentiment"].tolist()

In [8]:
test_text = df_music_dev["review_headline"].tolist()
test_text = [' '.join(t.split()[0:150]) for t in test_text]
test_text = np.array(test_text, dtype=object)[:, np.newaxis]
test_label = df_music_dev["sentiment"].tolist()

In [9]:
GPU_OPTIONS = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
CONFIG = tf.ConfigProto(gpu_options=GPU_OPTIONS)
sess = tf.Session(config = CONFIG)

In [10]:
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [11]:
class ElmoEmbeddingLayer(layers.Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable = True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/3', trainable=self.trainable, name="{}_module".format(self.name))
        self.trainable_weights += tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)
        
    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result
    
    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [ ]:
input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = ElmoEmbeddingLayer()(input_text)
dense = layers.Dense(256, activation='relu')(embedding)
pred = layers.Dense(1, activation='sigmoid')(dense)

model = Model(inputs=[input_text], outputs=pred)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(train_text, 
          train_label,
          validation_data=(test_text, test_label),
          epochs=5,
          batch_size=32)



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
elmo_embedding_layer_1 (Elmo (None, 1024)              4         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 262,661
Trainable params: 262,661
Non-trainable params: 0
_________________________________________________________________



Train on 529536 samples, validate on 176512 samples
Epoch 1/5

